In [1]:
file_path = '/content/KnowledgeDocument(pan_card_services).txt'
try:
    with open(file_path, 'r') as file:
        content = file.read()
        # print(content)
except FileNotFoundError:
    print("File not found.")
except IOError:
    print("Error reading the file.")

In [2]:
content

"# About Pan Card\n\n### What is Pan card?\n\nThe PAN card is a unique ten-digit alphanumeric identification number that is issued by the Income Tax Department of India to track the tax-related transactions of individuals and entities. The PAN card is mandatory for any financial transaction in India, including opening a bank account, buying or selling property, and filing income tax returns.\n\n### Who needs a Pan card?\n\nAll individuals/non-individuals (including foreign citizens/entities) earning taxable income in India\xa0must have a PAN card.\n\n### Types of PAN cards\n\nIn India, two types of PAN cards are available: e-PAN card and physical PAN card.\n\n1. e-PAN card: An e-PAN card is a digitally-signed PAN card issued in electronic format. It contains the same PAN details as a physical PAN card but is available in a digital format. It can be downloaded online and used as a valid identification document for various purposes. The e-PAN card is usually issued in a PDF format.\n2. P

In [3]:
def convert_to_title_and_passage(text):
    title = None
    passages = []

    lines = text.split("\n")
    current_passage = ""

    for line in lines:
        if line.startswith("# "):
            # Save the previous passage if exists
            if title is not None and current_passage.strip() != "":
                passages.append((title, current_passage.strip()))
                current_passage = ""

            # Extract the title
            title = line[2:].strip()
        else:
            current_passage += line + "\n"

    # Save the last passage
    if title is not None and current_passage.strip() != "":
        passages.append((title, current_passage.strip()))

    return passages

# Example usage:
document_text = content

title_passage_pairs = convert_to_title_and_passage(document_text)
for title, passage in title_passage_pairs:
    print("Title:", title)
    print("Passage:", passage)
    print()


Title: About Pan Card
Passage: ### What is Pan card?

The PAN card is a unique ten-digit alphanumeric identification number that is issued by the Income Tax Department of India to track the tax-related transactions of individuals and entities. The PAN card is mandatory for any financial transaction in India, including opening a bank account, buying or selling property, and filing income tax returns.

### Who needs a Pan card?

All individuals/non-individuals (including foreign citizens/entities) earning taxable income in India must have a PAN card.

### Types of PAN cards

In India, two types of PAN cards are available: e-PAN card and physical PAN card.

1. e-PAN card: An e-PAN card is a digitally-signed PAN card issued in electronic format. It contains the same PAN details as a physical PAN card but is available in a digital format. It can be downloaded online and used as a valid identification document for various purposes. The e-PAN card is usually issued in a PDF format.
2. Physica

#Chunk Article

DPR is based on BERT, so we obey to BERT's 512 token limit

In [4]:
print('before spillting, {:,} articles. \n'.format(len(content)))

passages_title = []
passages = []

print('Splitting....')



for i in range(0, len(title_passage_pairs)):

    title,passage = title_passage_pairs[i]

    words = passage.split()
    for i in range(0,len(words),100):

        chunk_words = words[i: i+100]
        chunk = " ".join(chunk_words)
        chunk = chunk.strip()
        if len(chunk)==0:
            continue

        passages.append(chunk)
        passages_title.append(title)

print('   Done.\n')

chunked_corpus = {'title':passages_title, 'text': passages}
print('After Splitting {:,}"passages"'.format(len(chunked_corpus['text'])))



before spillting, 17,879 articles. 

Splitting....
   Done.

After Splitting 31"passages"


# Create DPR Embedding

Now we feed these passages to out BERT Model

As Always with BERT, There are two steps:
1. The Tokenizer will break the passages into tokens and replace them with token IDs

2. The Encoder will feed each passage's token IDs through the BERT-type model to generateembedding for each passage.

# Tokenizer

In [5]:
!pip install transformers
from transformers import DPRContextEncoderTokenizerFast

In [6]:
from transformers import DPRQuestionEncoderTokenizer
#load the tokenizer
ctx_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

In [37]:
#We're passing the tokenizer
#   1. The passages
#   2. Parameters fimiliar from BERT Tokenizers

num_passages = len(chunked_corpus['text'])

print('Tokenizing {:,} passages fro DPR....'.format(num_passages))

#Tokenize the whole dataset
outputs = ctx_tokenizer(
    chunked_corpus['title'],
    chunked_corpus['text'],
    truncation = True,
    padding='longest',
    return_tensors = "pt"
)

print("done")

#'input_ids' holds the encoded tokens for the enntire corpus
input_ids = outputs["input_ids"]


Tokenizing 31 passages fro DPR....
done


In [38]:
print(input_ids.shape)

torch.Size([31, 287])


This tensor size tell us about the longest passsage is 279 tokens.

When retrieving reference text, the DPR model has seperate encoder for the question text and the reference text.

one implication of this is that the question and reference can be upto 512 tokens each

# "Encoding"(Generate Embedding)

In [39]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")

    print('There are %d GPU available' % torch.cuda.device_count())
    print('We will use the GPU: ', torch.cuda.get_device_name(0))

else:
    print('No GPU Available')

There are 1 GPU available
We will use the GPU:  Tesla T4


No we can load the encoder, and move it into the GPU. This will put the model from the huggingface repository.

In [40]:
#Import DPR encoder,the model that will generate the embedding
from transformers import DPRContextEncoder

ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

#Move the encoder model to the GPU
ctx_encoder = ctx_encoder.to(device=device)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Encoding can be slow, so we'll print the progress updates as we go. The below helper function just formats elapsed time(in second) as hh:mm:ss

In [41]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''

    #Round to nearest second
    elapsed_rounded = int(round(elapsed))

    #Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

With the model loaded, it's time to apply it to our tokenized passages.

In [42]:
import time
import math

#We're running the model forward only so no need for gradient
torch.set_grad_enabled(False)

#Track time elapsed for progress update
t0 = time.time()

#Track the current batch number, also for progress update
step = 0

#How many passages to process each batch
batch_size = 4

#Get the number of passages in the dataset, we'll use this in a few place
num_passages = input_ids.size()[0]

#Calculate the number of batches in the dataset
num_batches = math.ceil(num_passages/batch_size)

#As we embed the passages in batches, accumulate them in this list.
embed_batches = []

print('Generating embedding for {:,} passages....'.format(num_passages))

#For each batch of passages....
for i in range(0,num_passages, batch_size):

    #progree update after every 100 batches
    if step %1== 0 and not step ==0:
        #Calculate the elapsed time in minute
        elapsed = format_time(time.time() - t0)

        #Report Progress
        print('Batch{:>5} of {:>5}. Elapsed{:}'.format(step,num_batches,elapsed))

    #Select the nect batch
    batch_ids = input_ids[i:i+4,:]

    #Move them to the GPU
    batch_ids = batch_ids.to(device)

    #Run the encoder!
    output = ctx_encoder(
        batch_ids,
        return_dict = True
    )

    # The embedding are in the field "pooler_output"
    embeddings = output["pooler_output"]

    #Bring the embedding back over from the GPU and convert to numpy(out of pytorch)
    embeddings = embeddings.detach().cpu().numpy()

    embed_batches.append(embeddings)

    step +=1

print("Done.")

Generating embedding for 31 passages....
Batch    1 of     8. Elapsed0:00:00
Batch    2 of     8. Elapsed0:00:00
Batch    3 of     8. Elapsed0:00:00
Batch    4 of     8. Elapsed0:00:00
Batch    5 of     8. Elapsed0:00:00
Batch    6 of     8. Elapsed0:00:00
Batch    7 of     8. Elapsed0:00:01
Done.


A final step is to combine the result across all other batches

We can print the final matrix size and see that the length of embeddings is 786, which is the embedding length for BERT-base

In [43]:
import numpy as np

#Combine the results across all batches
embeddings = np.concatenate(embed_batches,axis=0)

print('Size of dataset embeddings: ', embeddings.shape)

Size of dataset embeddings:  (31, 768)


# Faiss index

Now that we have our reference passages all converted into embeddings, the next piece is performing the similiarity search between the question embedding and all of the reference embedding.


**Facebook AI Similiarity Search (FAISS)** is a library for performing KNN search.


The main feature of FAISS is that it implements a number of **Approximate Nearest Neaighbour(ANN)** techniques.Knn can be extremely computationally expensive to do "brute force", so ANN techniques attempt to address this by approximating the search ans sacrificing accuracy for speed. They also require that you first build an index for the collection of vector that you'll be searching.



###Create index

In [44]:
!pip install faiss-gpu

In [45]:
import faiss

# The dimension of embeddings to pass to the Hierarchical navigable Small Worlds(HNSW) Faiss Index
dim =768

#The number of bi directional links created for every new element during the HNSW Construction
m=64

# Let's use the Faissimplementation of HNSW for fast approximate nearest neighbour search
index = faiss.IndexHNSWFlat(dim, m, faiss.METRIC_INNER_PRODUCT)

There are 2 steps to building the index. We call it train and add.

because train takes less time than add, I feel add is the function which actually build the index.

In [46]:
print("Building the FAISS index....")

#Track elapsed time for progress update
t0= time.time()
index.train(embeddings)
index.add(embeddings)

print("DONE.")
print("Adding embedding to index took",format_time(time.time()-t0))

Building the FAISS index....
DONE.
Adding embedding to index took 0:00:00


#Example Search

Now we're ready to search.
When we use the complete RAG system to answer questions, RAG will handle this search step for us internally.

For the sake of illustration how DPR works,though, let's try a search of our own!



**DPR QUESTION ENCODER**

to perform a search against our DPR Embedding, we'll need to encode our query using DPR Question encoder rather than the context encoder we used earlier. DPR trains seperate encoders for the questions and reference text

In [47]:
from transformers import DPRQuestionEncoder

q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

#Move the model to GPU
q_encoder = q_encoder.to(device=device)

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We will also need question tokenizer

In [48]:
from transformers import DPRQuestionEncoderTokenizerFast
#load the tokenizer
q_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

####DEMO

In [49]:
#Tokenize the question
input_ids = q_tokenizer.encode("Where can I apply for PAN?",return_tensors = "pt")

#Move the model to GPU
input_ids = input_ids.to(device)

#Run the question the BERT and generate the question embedding
outputs = q_encoder(input_ids)

#The embedding stored in the 'pooler_output' property
q_embed = outputs['pooler_output']

#Our FAISS index is no GPU not on CPU. So move!!
q_embed = q_embed.cpu().numpy()

#checking embedding size
print("Query embedding: ", q_embed.shape)

Query embedding:  (1, 768)


We are ready to perform the search using the FAISS Index!

We call index.search, giving it our question embedding and the number of top matches that we want returned.

The search function returns the indices of the closest matching passages as well as the distances, similiarity between the query and each of those result.

In [50]:
#Find the k=3 most similiar passages to the question embedding'q_embed'
D,I = index.search(q_embed, k=3)

# Print out the indices and the similiarity scores
print("Closest matching indices:", I)
print("Inner Product", D)

Closest matching indices: [[10 29  6]]
Inner Product [[76.85603 74.05934 72.56772]]


The indices are not interesting to look, so let's map those back to the original passage text. We'll use the textwrap library to print the passage nicely.

In [51]:
len(chunked_corpus['text'])

31

In [52]:
import textwrap

#Wrap text to 80 character
wrapper = textwrap.TextWrapper(width=80)

#For each of the 'k' results
for i in I[0]:
    print("index", i)

    #Retrieve passage
    passage = chunked_corpus['text'][i]

    print("Passage:")
    print(wrapper.fill(passage))

    print('')

index 10
Passage:
to update the address on PAN Card - Passport - OCI Card - Bank Account Statement
in the country of residence - NRE Bank Account Statement in India - Residential
Permit ###
******************************************************************Process to
change the citizenship on PAN
Card****************************************************************** No direct
provisions exist to change citizenship in a Pan Card as the **PAN Card** doesn’t
display your citizenship. It is a document required to file taxes, carry out
investments and do transactions in India, whether you are a citizen, NRI, or
OCI. To change the citizenship in a PAN card, you must meet and notify your
jurisdictional Assessing Officer.

index 29
Passage:
visit their website for the payment process. The payment link provided by ABC
can be accessed directly to make the payment. **Can I apply for pan card without
Aadhaar?** Yes, NRIs can apply for a PAN card without an Aadhaar Card? They can
simply do so by fil

#ASK Question

In the pat few sections, we've looked closely at just the "Retriever Protion" of the RAG Framework. In this section, we'll use the full RAG model to generate answer to generate answers

###Create the Models Objects

#####Huggingface Dataset

RAG requires that we place our reference data into a Dataset object.



In [53]:
!pip install datasets

In [54]:
from datasets import Dataset
import pandas as pd

#Create DataFrame from the Dictionary
df = pd.DataFrame(chunked_corpus)

#Convert the DataFrame into huggingface Dataset object.
dataset = Dataset.from_pandas(df)

#Check out the object
print(dataset)

Dataset({
    features: ['title', 'text'],
    num_rows: 31
})


Next,we need to add the passages embeddings. We still have these in numpy matrix named embedding, but they actually need to be a list of arrays.

In [55]:
embs = []

#For each embeddings..
for i in range(embeddings.shape[0]):

    #Add it to the list
    embs.append(embeddings[i,:])

In [56]:
len(embs)

31

In [57]:
#Add the embeddings to the dataset as a new column
#This is not done in place, so make sure to capture the return value.
dataset = dataset.add_column("embeddings", embs)

#Display the dataset content
dataset

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 31
})

In [58]:
dataset.add_faiss_index(column="embeddings",index_name="embeddings",custom_index=index,faiss_verbose=True)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 31
})

Now our reference dataset is all set

##Retriever

The model we are using is called rag-sequence-nq.

There are 2 variants of RAG: RAG-Sequence and RAG-Token.

RAG-Sequence combine information from multiple passages.

RAG-Token is capable of combining information. It generates single answer to the question. All documents are taken into consideration as we generate each token for this answer.

In [59]:
!pip install datasets
!pip install faiss-cpu

In [60]:
from transformers import RagRetriever

retriever = RagRetriever.from_pretrained(
    "facebook/rag-sequence-nq",

    use_dummy_dataset = True, #The dummy dataset is reduced-size Wikipedia

    indexed_dataset = dataset, #pass in our own dataset
    index_name = "embeddings", #Specify the name of FAISS index
    )

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

##Tokenizer

load a takenizer to tokenize the question

the RagTokr=enizer contains multiple tokenizers:


*   The question tokenizer is accessed with the question_encoder funtion

*   Presumably the generator, which is BART, has its own tokenizer that it's using internally to tokenize the passage returned by the retriever.



In [61]:
from transformers import RagTokenizer

tokenizer = RagTokenizer.from_pretrained(
    "facebook/rag-sequence-nq"
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

##Generator

We pass the retriever object to the generator object, so we'll see that all of our interaction with RAG is actually just handled through this Generator instance.

In [62]:
from transformers import RagSequenceForGeneration

model = RagSequenceForGeneration.from_pretrained(
    "facebook/rag-sequence-nq",
    retriever =retriever
)

Some weights of the model checkpoint at facebook/rag-sequence-nq were not used when initializing RagSequenceForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.weight', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing RagSequenceForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagSequenceForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#Example Question

In [ ]:
import time
import torch

t0 = time.time()


question = "What is PAN?"
#Tokenize nthe question
input_ids = tokenizer.question_encoder(question)['input_ids']

input_ids = torch.tensor(input_ids).unsqueeze(0)
#Give the question to RAG have have it generate an answer
generated = model.generate(input_ids)

#Convert the answer tokens back inpt a single string
generated_string = tokenizer.batch_decode(generated, skip_special_token=True)[0]

print("Q: " + question)
print("A: " + generated_string)

print("\n Response took %.2f seconds" %(time.time() - t0))